# Notebook 3: Model building and evaluation

This notebook entails the building and evaluation of the machine learning models introduced in section 3.2 of the paper. Based on the tuned variants of the learning algorithms, their explainability is evaluated and compared.
The results are presented in the paper in section 4.3.

**Table of Contents**:

0. [Technical setup](#setup)
1. [Load data and define functions](#setup)
2. [Build, evaluate and explain machine learning models](#models)
    1. [Random Forest](#rf)
    2. [Support Vector Machine](#svm)
    3. [XGBoost](#xgb)
    4. [LSTM](#lstm)
3. [Performance comparison](#comp)

# 3.0 Technical setup <a id="setup"></a>

In [ ]:
!pip install gensim==4.0.1
!pip install shap
!pip install lime

from google.colab import drive
drive.mount('/content/drive')

In [5]:
# import modules
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import Counter
import string
from copy import deepcopy
import seaborn as sns
from tqdm.notebook import tqdm
import statistics
import warnings
import random
from scipy import stats

from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedKFold, cross_val_score, cross_validate, RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

from gensim.models import FastText

from lime import lime_text
from lime.lime_text import LimeTextExplainer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
from keras.callbacks import EarlyStopping

from xgboost import XGBClassifier

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
# define functions for saving and loading pickled objects
def save_pickle(objectname, picklename):
    pickle_out = open(picklename,"wb")
    pickle.dump(objectname, pickle_out)
    pickle_out.close()
    print(picklename, 'successfully pickled.') 
    
def load_pickle(picklename):
    pickle_in = open(picklename,"rb")
    return pickle.load(pickle_in)

# 3.1 Load data and define functions <a id="load"></a>

In [9]:
text = load_pickle("/content/drive/MyDrive/Seminar/hateXplain_processed.pickle")
X = load_pickle("/content/drive/MyDrive/Seminar/data_corpus.pickle")
Y = load_pickle("/content/drive/MyDrive/Seminar/labels.pickle")
rationales = load_pickle("/content/drive/MyDrive/Seminar/rationales.pickle")
    
# check for correct lengths
print(X.shape, type(X))
print(len(Y), type(Y))
print(len(rationales), type(rationales))

# load embeddings model
text_model = FastText.load('/content/drive/MyDrive/Seminar/model1.bin')

(20147, 300) <class 'numpy.ndarray'>
20147 <class 'pandas.core.series.Series'>
20147 <class 'pandas.core.series.Series'>


In [10]:
# function to create a sentence vector based on a list of tokens with 
# the defined embeddings model
def sentence_vector(sentence, d):
    X = np.zeros([len(sentence), d])
    for i in range(len(sentence)):
        wv = text_model.wv[sentence[i]]
        norm_wv = np.linalg.norm(wv)
        with np.errstate(invalid='ignore'):
            X[i] = wv/norm_wv
            
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        sen_vector = np.nanmean(X, axis = 0)
           
    return sen_vector

# function to evaluate the explainability of the respective model using LIME
# the mean explainability measure among the n posts to be evaluated is returned
def explain_clf(n, classifier = "Not LSTM"):
    explainer = LimeTextExplainer(class_names=classes)
    results = []

    for i in tqdm(range(len(ind_rat[:n])), "Explaining Predictions"):
        n_rat = sum(text["rationales_comb"][ind_rat[i]]) #* 2
        if classifier == "LSTM":
          exp = explainer.explain_instance(" ".join(text["tokens_processed"][ind_rat[i]]), 
                                           predict_LSTM, 
                                           num_features = n_rat)
          
        else:
          exp = explainer.explain_instance(" ".join(text["tokens_processed"][ind_rat[i]]), 
                                           predict_clf, 
                                           num_features = n_rat)

        res = exp.as_list()
        results.append(eval_explanation(res, i))

    return statistics.mean(results)


# function to predict the class probabilities based on texts and a trained clf
# by retrieving the sentence_vector and using .predict_proba()
def predict_clf(texts):
    d = len(text_model.wv['x'])
    text_data_pred = np.zeros([len(texts),len(text_model.wv['x'])])
    for i in range(len(texts)):
        tokens = texts[i].split()
        text_data_pred[i, :] = sentence_vector(tokens, d)
    text_data_pred = np.nan_to_num(text_data_pred)
    pred = clf.predict_proba(text_data_pred)
    return pred


# function to predict the class probabilities based on texts and a trained LSTM
# model by converting the texts into padded sequences ans using .predict()
def predict_LSTM(texts):
  x = tokenizer.texts_to_sequences(texts)
  x = pad_sequences(x, sequence_length)
  pred = LSTM_model.predict(x) # for one prediction of index 0 only: predict(X_padded[:1])
  return np.c_[ pred, 1-pred ] 


# function to evaluate the explanations provided by LIME, specifically,
# the relative relationship of correct identified rationales to overll correct
# rationales is returned
def eval_explanation(res_obj, ind):
    rat = text["rationales_comb"][ind_rat[ind]]
    correct_rat = [i for i, x in enumerate(rat) if x == 1]
    
    tokens = text["tokens_processed"][ind_rat[ind]]
    try:
      pred_rat = [tokens.index(res_obj[i][0]) for i in range(len(res_obj))]
      correct_pred = [i for i in pred_rat if i in correct_rat]    
      return len(correct_pred)/len(correct_rat)

    except ValueError:
      return 0

# function to plot the confusion matrix of the cross-validation
def plot_cm(cm):
  df_cm = pd.DataFrame(np.around(cm/np.sum(cm), 2), range(len(set(Y))), range(len(set(Y))))
  df_cm.index.name = 'True'
  df_cm.columns.name = 'Predicted'
  ax = plt.axes()
  sns.set(font_scale=1.4) # for label size
  sns.heatmap(df_cm, annot=True, annot_kws={"size": 10}, ax = ax, fmt='g')

# 3.2 Build, evaluate and explain machine learning models <a id="models"></a>

In [ ]:
# collect indices which contain rationales & randomly shuffle them
random.seed(0)
ind_rat = [i for i in range(text.shape[0]) if len(text.rationales_comb[i]) > 2] 
random.shuffle(ind_rat)

# define names of classes
classes = ["normal", "HateSpeech"]

# construct 5 shuffled and stratified folds for evaluating clssifier performance
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42)

# function to return the name of object
def namestr(obj):
    return [name for name in globals() if globals()[name] is obj]

## 3.2.1 Random Forest <a id="rc"></a>

In [ ]:
# parameter tuning of a random forest classifier via grid search

from sklearn.model_selection import GridSearchCV
parameters = {'max_depth':[50, 90, 150], "n_estimators":[50, 100, 130], "min_samples_leaf":[2, 10], 
              "max_samples":[0.3, 0.5], "min_samples_split":[10, 20], 'bootstrap':(True, False)}
              
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters)
clf.fit(X, Y)

clf.best_params_

In [ ]:
# evaluating the performance of the tuned random forest classifier by 5-fold 
# cross-validation

rf_best = RandomForestClassifier(bootstrap=True, class_weight='balanced',criterion='gini', 
                                 max_depth=90, max_features='auto', max_samples=0.3,                                   
                                 min_samples_leaf=2, min_samples_split=10, n_estimators=130,                                   
                                 n_jobs=-1, random_state=42)

acc = []
rf_cm = np.zeros((2, 2))

for train_ix, test_ix in kfold.split(X, Y):
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = Y[train_ix], Y[test_ix]
    clf_deep = deepcopy(rf_best)
    clf_new = clf_deep.fit(train_X, train_y)
    pred_new = clf_new.predict(test_X)

    cm = confusion_matrix(test_y, pred_new)
    rf_cm += cm
    acc.append(accuracy_score(test_y, pred_new))

print('Accuracy:', np.mean(acc))
print(acc)

plot_cm(rf_cm)

In [ ]:
# explain the predictions of the tuned random forest classifier

clf = RandomForestClassifier(bootstrap=True, class_weight='balanced',criterion='gini', 
                             max_depth=90, max_features='auto', max_samples=0.3,     
                             min_samples_leaf=2, min_samples_split=10, n_estimators=130,   
                             n_jobs=-1, random_state=42)
clf = clf.fit(X, Y)
explain_clf(500)

In [ ]:
# visualization of LIME applied to the first post using random forest

explainer = LimeTextExplainer(class_names=["Normal", "Hate Speech"])
exp = explainer.explain_instance(" ".join(text["tokens_processed"][ind_rat[0]]), new_predict, num_features=6)
exp.show_in_notebook(text=True)

## 3.2.2 SVM <a id="svm"></a>

In [ ]:
# parameter tuning of a SVM classifier via grid search

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100]}
              
svm = SVC(max_iter=-1, probability=True, tol=0.001)
clf = GridSearchCV(svm, parameters)
clf.fit(X, Y)

clf.best_params_

In [ ]:
# evaluating the performance of the tuned SVM classifier by 5-fold cross-validation

svc_best = SVC(C=100, kernel='rbf', max_iter=-1, random_state=42)

acc = []
svm_cm = np.zeros((2, 2))

for train_ix, test_ix in kfold.split(X, Y):
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = Y[train_ix], Y[test_ix]

    clf_deep = deepcopy(svc_best)
    clf_new = clf_deep.fit(train_X, train_y)
    pred_new = clf_new.predict(test_X)

    cm = confusion_matrix(test_y, pred_new)
    svm_cm += cm

    acc.append(accuracy_score(test_y, pred_new))

print('Accuracy:', np.mean(acc))
print(acc)

plot_cm(svm_cm)

In [ ]:
# explain the predictions of the tuned SVM classifier

clf = SVC(C=100, kernel='rbf', max_iter=-1, random_state=42, probability=True)

clf = clf.fit(X, Y)
explain_clf(500)

## 3.2.3 XGBoost <a id="xgb"></a>

In [13]:
# create own class due to adaption

class MyXGBClassifier(XGBClassifier):
    @property
    def coef_(self):
        return None

In [ ]:
# parameter tuning of a XGBoost classifier via grid search

parameters = {'learning_rate':[0.3, 0.6], 'max_depth':[100, 150], 'n_estimators':[100, 150, 300],
              'reg_lambda':[1, 1.5], 'gamma':[0.2, 0.5], 'reg_alpha':[0.25, 0.5]}
              
xgb = MyXGBClassifier(booster = 'gbtree', objective= "binary:logistic", use_label_encoder=False,
                      eval_metric = 'logloss')

clf = GridSearchCV(xgb, parameters)
clf.fit(X, Y)

clf.best_params_

In [ ]:
# evaluating the performance of the XGBoost classifier with optimal
# parameters evaluated by grid search

xgb_best = MyXGBClassifier(learning_rate=0.3, booster = 'gbtree', objective= "binary:logistic", use_label_encoder=False, 
                           max_depth = 100, gamma = 0.2, reg_alpha = 0.25, reg_lambda = 1.5, eval_metric = 'logloss', 
                           n_estimators=300, random_state = 42, n_jobs = -1)
                     
acc = []
xgb_cm = np.zeros((2, 2))

for train_ix, test_ix in kfold.split(X, Y):
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = Y[train_ix], Y[test_ix]
    clf_deep = deepcopy(xgb_best)
    clf_new = clf_deep.fit(train_X, train_y)
    pred_new = clf_new.predict(test_X)
    
    cm = confusion_matrix(test_y, pred_new)
    xgb_cm += cm
    f1.append(f1_score(test_y, pred_new, average = 'binary', labels=np.unique(test_y)))
    acc.append(accuracy_score(test_y, pred_new))

print('Accuracy:', np.mean(acc))
print(acc)

plot_cm(xgb_cm)

In [14]:
# explain the predictions of the tuned XGBoost classifier

clf = MyXGBClassifier(learning_rate=0.3, booster = 'gbtree', objective= "binary:logistic", use_label_encoder=False, 
                           max_depth = 100, gamma = 0.2, reg_alpha = 0.25, 
                           reg_lambda = 1.5, eval_metric = 'logloss', n_estimators=300, random_state = 42, n_jobs = -1)

clf = clf.fit(X, Y)

explain_clf(500)

In [ ]:
# evaluating the randomness of LIME explanations by repeating the explanation of the same 50
# posts 30 times and computing the variance of the explainability scores 

exp_scores = []
for i in range(30):
  exp_scores.append(explain_clf(50))
print(np.var(exp_scores))

## 3.2.4 LSTM <a id="lstm"></a>

In [27]:
# setup LSTM architecture to return a model which can be trained subsequently
def get_LSTM():
  model = Sequential()
  model.add(Embedding(num_words, # add embedding layer based on the trained embeddings model
                      embedding_dim,
                      embeddings_initializer=Constant(embedding_matrix),
                      input_length=sequence_length,
                      trainable=False))
  model.add(SpatialDropout1D(0.2)) # include dropout layer to counteract overfitting
  model.add(Bidirectional(LSTM(300, return_sequences=True)))
  model.add(Bidirectional(LSTM(150)))
  model.add(Dropout(0.25))
  model.add(Dense(units=1, activation='sigmoid')) 
  model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['acc'])
  print(model.summary())
  return model

# define function to print the history of the training of the LSTM network with
# regard to accuracy and loss
def show_plt():
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

In [15]:
# determine lengths of post of post corpus
length_posts = [len(text["tokens_processed"][i]) for i in range(text.shape[0])]

batch_size = 64
sequence_length = max(length_posts) # define maximum length of post for subsequent padding 
embedding_dim = 300 # as defined in the embeddings model

texts = text['tokens_processed']

# tokenize the processed tokens such that they can be easily padded and an index
# with regard to their position can be returned
tokenizer = Tokenizer(split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(texts.values)
word_index = tokenizer.word_index
X_token = tokenizer.texts_to_sequences(texts.values)
X_padded = pad_sequences(X_token, sequence_length) # pad the sequences so they are all the same length (sequence_length)

# define number of words as the true number plus one to account for padding
num_words = len(word_index) + 1

# initialize the embedding matrix with zeros
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in the tokenizer, find the vector in the embeddings model
for word, i in word_index.items():
    embedding_vector = text_model.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector # add vector to matrix
    else:
        # if word is not in model (which should not be the case):
        # assign random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

In [ ]:
# evaluating the performance of the LSTM network by 5-fold cross-validation

acc = []
lstm_cm = np.zeros((2, 2))

for train_ix, test_ix in kfold.split(X_padded, Y):    
  train_X, test_X = X_padded[train_ix], X_padded[test_ix]
  train_y, test_y = Y[train_ix], Y[test_ix]

  LSTM_model = get_LSTM()
  history = LSTM_model.fit(train_X, train_y, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.3, callbacks=[EarlyStopping(monitor='acc', min_delta=0.01, patience=2, restore_best_weights = True)])
  #show_plt()
  pred_new = LSTM_model.predict(test_X)
  
  cm = confusion_matrix(test_y, np.around(pred_new))
  lstm_cm += cm
  f1.append(f1_score(test_y, np.around(pred_new), average = 'binary', labels=np.unique(test_y)))
  acc.append(accuracy_score(test_y, np.around(pred_new)))


print('Accuracy:', np.mean(acc))
print(acc)

plot_cm(lstm_cm)

In [ ]:
# explain the predictions of the LSTM network

LSTM_model = get_LSTM()
history = LSTM_model.fit(X_padded, Y, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.3, callbacks=[EarlyStopping(monitor='acc', min_delta=0.01, patience=2, restore_best_weights = True)])

explain_clf(500, classifier = "LSTM")

# 3.3 Performance comparison <a id="comp"></a>

In [50]:
# store accuracy - and explainability score of respective classifiers

rf_acc, rf_exp = 0.6742, 0.6511
svm_acc, svm_exp = 0.7109, 0.6668
xgboost_acc, xgboost_exp = 0.6928, 0.6617
lstm_acc, lstm_exp = 0.7123, 0.6931

x = [rf_acc, svm_acc, xgboost_acc, lstm_acc]
y = [rf_exp, svm_exp, xgboost_exp, lstm_exp]

In [ ]:
# quantify relationship between accuracy and explainability for the analyzed models

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
print(p_value)
print(slope)

In [ ]:
# plot predictive performance against explainability performance

coef = np.polyfit(x,y,1)
poly1d_fn = np.poly1d(coef) 

colors=["red", "blue", "green", "black", "orange"]

fig = plt.figure()
ax = fig.add_subplot(111)

for i in range(len(x)):
    ax.scatter(x[i], y[i], color=colors[i])

ax.grid(False)
ax.set(facecolor = "white")

plt.legend(["Random forest", "SVM", "XGBoost", "LSTM"], loc=2, bbox_to_anchor=(1.05, 1), borderaxespad=0., fontsize=11)
plt.plot(x, poly1d_fn(x), '--k')

plt.xlabel("Accuracy", fontsize=12)
plt.ylabel("Explainability", fontsize=12)
plt.xticks(fontsize=12)
plt.show()